In [1]:
import pandas as pd 
from pandas.tools.plotting import scatter_matrix
import numpy as np
from math import radians, cos, sin, acos, sqrt
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import statsmodels.api as sm
from patsy import dmatrices
import scipy.stats as stats
import datetime
from datetime import date
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from subprocess import check_output

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
train=pd.read_csv("/Users/jesminew/Desktop/train.csv")

In [4]:
train.drop(['Field_1', 'Unnamed: 0'], axis=1, inplace=True)

#train['srch_ci']= pd.to_datetime(train['srch_ci'],errors='coerce')
#train['srch_co'] = pd.to_datetime(train['srch_co'],errors='coerce')
train.drop(['srch_ci', 'srch_co'], axis=1, inplace=True)
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,year,month,duration,srcbeforeci,cimonth
0,2014-01-17 06:24:56,2,3,66,318,22418,420.6642,756,0,1,...,5,2,50,191,18,2014,1,3.0,89.0,4.0
1,2014-01-18 14:33:31,2,3,66,318,52078,NaN,756,0,1,...,1,2,50,191,28,2014,1,3.0,87.0,4.0
2,2014-01-21 06:39:08,2,3,66,318,22418,421.8551,756,0,1,...,1,2,50,191,25,2014,1,3.0,85.0,4.0
3,2014-01-21 06:40:18,2,3,66,318,22418,421.8551,756,0,1,...,1,2,50,191,25,2014,1,2.0,86.0,4.0
4,2014-01-22 06:10:02,2,3,66,318,22418,421.8551,756,0,1,...,1,2,50,191,25,2014,1,2.0,85.0,4.0


In [27]:
len(train)

249322

In [7]:
#train=train.drop('date_time',axis=1)
train.fillna(0, inplace=True)

In [75]:
test = pd.read_csv("/Users/jesminew/Desktop/test.csv")
test.drop(['Field_1', 'Unnamed: 0'], axis=1, inplace=True)
test.drop(['srch_ci', 'srch_co','date_time'], axis=1, inplace=True)
test.fillna(0, inplace=True)


In [65]:
aggs = train.groupby(['srch_destination_id',
                         'hotel_cluster','hotel_country','hotel_market'])['is_booking'].agg(['sum','count'])
aggs

sum  count
srch_destination_id hotel_cluster hotel_country hotel_market            
4                   22            7             246             1      2
                    82            7             246             1      2
8                   7             50            416             0      3
                    16            50            416             2      6
                    77            50            416             1      2
9                   20            150           2101            0      1
                    61            150           2101            0      1
                    75            150           2101            0      2
11                  42            50            824             0      1
                    47            50            824             0      1
                    51            50            824             0      1
                    94            50            824             1      3
14                  20            27            1434            1      3
16                  7             50            419             1      3
                    76            50            419             0      1
18                  63            8             109             0      1
19                  3             102           1522            0      1
                    20            102           1522            0      2
                    60            102           1522            0      1
                    61            102           1522            0      4
                    64            102           1522            0      5
                    75            102           1522            4     14
                    78            102           1522            0      1
                    89            102           1522            0      1
                    93            102           1522            0      1
21                  3             77            1454            0      3
                    5             77            1454            1      7
                    7             77            1454            1      3
                    12            77            1454            0      1
                    25            77            1454            0      1
...                                                           ...    ...
63951               43            22            301             1      2
63994               2             77            21              0      1
64059               10            50            1664            0      1
                    63            50            1664            1      2
64113               97            70            19              1      2
64385               2             70            309             0      2
                    28            70            309             0      1
                    90            70            309             0      1
64415               28            50            529             1      2
64504               6             144           1450            0      1
                    57            144           1450            0      1
64560               2             68            275             0      1
                    37            68            275             0      1
                    98            68            275             0      1
64573               41            50            699             0      1
64627               9             50            1632            0      1
                    13            50            1632            1      2
                    16            50            1632            0      1
                    18            50            1632            0      1
                    33            50            1632            0      1
                    34            50            1632            0      1
                    42            50            1632            1      2
                    48            50           

In [68]:
agg = aggs.groupby(['srch_destination_id','hotel_cluster','hotel_country','hotel_market']).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
#success rate of click 
agg['relevance'] = 0.95*agg['bookings']+0.05*agg['clicks']


In [69]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets


In [77]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
srch_destination_id,
4,82 22
8,16 77 7
9,75 61 20
11,94 51 47 42
14,20


In [71]:
most_pop = agg.groupby(['hotel_country']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
hotel_country,
0,8 8 8 30 35
1,11 58 38 97 57
2,38 86 58 43 20
3,99 82 60
4,36 46 36 29 58


In [73]:
most_pop = agg.groupby(['hotel_market']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
hotel_market,
0,3 3 53 89 89
1,5 53
2,64 2 28 11 29
3,82 25 83 5 11
4,2 59 29 46 59


In [79]:
testdata=pd.DataFrame({'srch_destination_id':test.srch_destination_id})
testdata = testdata.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)

In [81]:
testdata

,srch_destination_id,hotel_cluster
0,8250,1 45 79 54 24
1,8250,1 45 79 54 24
2,8250,1 45 79 54 24
3,12193,95 18 50 94 28
4,12193,95 18 50 94 28
5,12193,95 18 50 94 28
6,8803,46 81 64 58 59
7,8803,46 81 64 58 59
8,8803,46 81 64 58 59
9,8803,46 81 64 58 59
